In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_MN = pd.read_html('https://uspdigital.usp.br/jupiterweb/listarGradeCurricular?codcg=45&codcur=45052&codhab=1&tipo=N',
                        match= "Disciplinas")

In [77]:
print(f'Total tables: {len(table_MN)}')

Total tables: 4


In [78]:
df = table_MN[3]
df.head()

,0,1,2,3,4,5,6,7
0,Disciplinas Obrigatórias,Disciplinas Obrigatórias,Disciplinas Obrigatórias,Disciplinas Obrigatórias,Disciplinas Obrigatórias,Disciplinas Obrigatórias,Disciplinas Obrigatórias,Disciplinas Obrigatórias
1,1º Período Ideal,1º Período Ideal,Créd. Aula,Créd. Trab.,CH,CE,CP,ATPA
2,MAC0101,Integração na Universidade e na Profissão,2,0,30,NaN,NaN,NaN
3,MAC0105,Fundamentos de Matemática para a Computação,4,0,60,NaN,NaN,NaN
4,MAC0110,Introdução à Computação,4,0,60,NaN,NaN,NaN


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       273 non-null    object
 1   1       273 non-null    object
 2   2       273 non-null    object
 3   3       273 non-null    object
 4   4       269 non-null    object
 5   5       48 non-null     object
 6   6       42 non-null     object
 7   7       42 non-null     object
dtypes: object(8)
memory usage: 18.5+ KB
